In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [18]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn, optim
from transformers import BertForSequenceClassification, BertTokenizer, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [19]:
tokenizer = BertTokenizer.from_pretrained("monologg/kobert")

In [11]:
train_df = pd.read_csv('/content/drive/MyDrive/NLP_commit/Comment_guard/unsmile_train_v1.0.tsv', delimiter='\t')
valid_df = pd.read_csv('/content/drive/MyDrive/NLP_commit/Comment_guard/unsmile_valid_v1.0.tsv', delimiter='\t')

In [21]:
# '문장' 열과 레이블 생성 (0: 부정적인 문장)
train_sentences = train_df['문장']
train_labels = torch.zeros(len(train_df), dtype=torch.long)

valid_sentences = valid_df['문장']
valid_labels = torch.zeros(len(valid_df), dtype=torch.long)


In [22]:
# 문장을 토크나이저로 변환하여 패딩 적용하는 함수 정의
def tokenize_sentences(sentences, max_length=50):
    tokens = tokenizer(sentences.tolist(), padding=True, truncation=True, max_length=max_length, return_tensors='pt')
    return tokens

In [ ]:
# 훈련 데이터와 검증 데이터 전처리
train_tokens = tokenize_sentences(train_sentences)
valid_tokens = tokenize_sentences(valid_sentences)

In [15]:
print(train_tokens)
print(valid_tokens)

{'input_ids': tensor([[2, 0, 0,  ..., 1, 1, 1],
        [2, 0, 0,  ..., 1, 1, 1],
        [2, 0, 0,  ..., 1, 1, 1],
        ...,
        [2, 0, 0,  ..., 1, 1, 1],
        [2, 0, 0,  ..., 1, 1, 1],
        [2, 0, 0,  ..., 1, 1, 1]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}
{'input_ids': tensor([[   2,    0, 7347,  ...,    1,    1,    1],
        [   2,    0,    0,  ...,    1,    1,    1],
        [   2,    0, 5782,  ...,    1,    1,    1],
        ...,
        [   2,    0,    0,  ...,    1,    1,    1],
        [   2,  497,    0,  ...,    1,    1,    1],
      